<a href="https://colab.research.google.com/github/liangsheng02/Butterfly-Voice-Changing-Bowtie/blob/master/fastai_trec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from torchtext import *
import pandas as pd
import torch
import fastai
from fastai.text import *
from fastai.callbacks import *
import random
import numpy as np

In [0]:
def random_seed(seed_value, use_cuda):
  np.random.seed(seed_value) # cpu vars
  torch.manual_seed(seed_value) # cpu  vars
  random.seed(seed_value) # Python
  if use_cuda:
    torch.cuda.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value) # gpu vars
    torch.backends.cudnn.deterministic = True  #needed
    torch.backends.cudnn.benchmark = False
random_seed(180127818, True)

In [0]:
#TREC dataset
# set up fields
TEXT = data.Field(lower=True, include_lengths=True, batch_first=True)
LABEL = data.Field(sequential=False)
# make splits for data
train, val = datasets.TREC.splits(TEXT, LABEL)
label_set = list(set([i.label for i in val.examples]))
label_dict = {label_set[i]:i for i in range(len(label_set))}
def dataset2df(dataset, label_dict):
  df = pd.DataFrame(columns=('label','text'))
  for i in range(len(dataset.examples)):
    df = df.append(pd.DataFrame({'label':[label_dict[dataset.examples[i].label]],'text':[" ".join(dataset.examples[i].text)]}),ignore_index=True)
  return df
train = dataset2df(train, label_dict)
val = dataset2df(val, label_dict)

In [0]:
path = '/content/drive/My Drive/trec/'
#epoch number, follow https://github.com/fastai/fastai/blob/master/examples/ULMFit.ipynb
lm_epoch = 10
cls_epoch = 2

In [0]:
# CREATE data
# Language model data bunch
data_lm = TextLMDataBunch.from_df('.', train, val, text_cols='text', label_cols='label')
data_lm.save(path+'data_lm.pkl')
# Classifier model data
data_clas  = TextClasDataBunch.from_df('.', train_df=train,text_cols='text',label_cols='label',valid_df=val,vocab=data_lm.train_ds.vocab)
data_clas.save(path+'data_clas.pkl')
# Classifier model data bwd
data_clas_bwd  = TextClasDataBunch.from_df('.', train_df=train,text_cols='text',label_cols='label',valid_df=val,vocab=data_lm.train_ds.vocab, backwards=True)
data_clas_bwd.save(path+'data_clas_bwd.pkl')

In [0]:
# LOAD data
#forward
data_lm = load_data(path, 'data_lm.pkl', bs=128, bptt=70)
data_clas = load_data(path, 'data_clas.pkl', bs=64)
#bwd
data_bwd = load_data(path, 'data_lm.pkl', bs=128, bptt=70, backwards=True)
data_clas_bwd = load_data(path, 'data_clas_bwd.pkl', bs=64)

In [8]:
#FWD
learn = language_model_learner(data_lm, AWD_LSTM,pretrained=True)
learn = learn.to_fp16(clip=0.1)
#The Learner object we get is frozen by default, which means we only train the embeddings at first (since some of them are random).
learn.fit_one_cycle(1, 1e-2, moms=(0.8,0.7), wd=0.1)
#unfreeze
learn.unfreeze()
#LM fine-tuning
learn.fit_one_cycle(lm_epoch, 1e-3, moms=(0.8,0.7), wd=0.1)
learn.save_encoder(path+'fwd_enc')
# Classifier Fine-tuning by gradual unfreezing 
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5, pretrained=False)
learn.load_encoder(path+'fwd_enc')
lr = 0.01
learn.fit_one_cycle(1, lr, moms=(0.8,0.7), wd=0.1)
learn.freeze_to(-2)
lr /= 2
learn.fit_one_cycle(1, slice(lr/(2.6**4),lr), moms=(0.8,0.7), wd=0.1)
learn.freeze_to(-3)
lr /= 2
learn.fit_one_cycle(1, slice(lr/(2.6**4),lr), moms=(0.8,0.7), wd=0.1)
# all
learn.unfreeze()
lr /= 5
learn.fit_one_cycle(cls_epoch, slice(lr/(2.6**4),lr), moms=(0.8,0.7), wd=0.1)

epoch,train_loss,valid_loss,accuracy,time
0,5.863532,4.161353,0.272321,00:05


epoch,train_loss,valid_loss,accuracy,time
0,5.090587,3.132791,0.478013,00:07
1,4.593792,2.828482,0.497768,00:07
2,4.312400,2.598358,0.533482,00:07
3,4.061215,2.472150,0.545089,00:07
4,3.883097,2.412254,0.562165,00:07
5,3.715351,2.349417,0.557589,00:07
6,3.571049,2.316364,0.562835,00:07
7,3.450858,2.305882,0.568862,00:07
8,3.363286,2.310164,0.563504,00:07
9,3.296240,2.306876,0.566964,00:07


epoch,train_loss,valid_loss,accuracy,time
0,0.874741,0.514782,0.818000,00:03


epoch,train_loss,valid_loss,accuracy,time
0,0.623411,0.314566,0.890000,00:04


epoch,train_loss,valid_loss,accuracy,time
0,0.458824,0.279786,0.904000,00:06


epoch,train_loss,valid_loss,accuracy,time
0,0.376816,0.268887,0.908000,00:08
1,0.337634,0.246458,0.920000,00:08


In [0]:
pred_fwd,lbl_fwd = learn.get_preds(ordered=True)
acc_fwd = accuracy(pred_fwd, lbl_fwd)
learn.save(path+'fwd_clas'+'_'+str(lm_epoch)+'_'+str(cls_epoch)+'_'+str(acc_fwd.item())[2:6])

In [0]:
# Bidirectional
learn_bwd = language_model_learner(data_bwd, AWD_LSTM, pretrained=True)
learn_bwd = learn_bwd.to_fp16(clip=0.1)
learn_bwd.fit_one_cycle(1, 1e-2, moms=(0.8,0.7), wd=0.1)
#unfreeze
learn_bwd.unfreeze()
#LM fine-tuning
learn_bwd.fit_one_cycle(lm_epoch, 1e-3, moms=(0.8,0.7), wd=0.1)
learn_bwd.save_encoder(path+'bwd_enc')
# graual unfreezing
learn_bwd = text_classifier_learner(data_clas_bwd, AWD_LSTM, drop_mult=0.5, pretrained=False)
learn_bwd.load_encoder(path+'bwd_enc')
lr = 0.01
learn_bwd.fit_one_cycle(1, lr, moms=(0.8,0.7), wd=0.1)
learn_bwd.freeze_to(-2)
lr /= 2
learn_bwd.fit_one_cycle(1, slice(lr/(2.6**4),lr), moms=(0.8,0.7), wd=0.1)
learn_bwd.freeze_to(-3)
lr /= 2
learn_bwd.fit_one_cycle(1, slice(lr/(2.6**4),lr), moms=(0.8,0.7), wd=0.1)
learn_bwd.unfreeze()
lr /= 5
learn_bwd.fit_one_cycle(cls_epoch, slice(lr/(2.6**4),lr), moms=(0.8,0.7), wd=0.1)
pred_bwd,lbl_bwd = learn_bwd.get_preds(ordered=True)
final_pred = (pred_fwd+pred_bwd)/2
acc_bwd = accuracy(final_pred, lbl_fwd)
learn_bwd.save(path+'bwd_clas'+'_'+str(lm_epoch)+'_'+str(cls_epoch)+'_'+str(acc_bwd.item())[2:6])

In [12]:
"""
learn = text_classifier_learner(data_clas, AWD_LSTM, pretrained=False)
learn.load(path+'fwd_clas')
pred_fwd,lbl_fwd = learn.get_preds(ordered=True)
accuracy(pred_fwd, lbl_fwd)
#
learn_bwd = text_classifier_learner(data_clas_bwd, AWD_LSTM, pretrained=False)
learn_bwd.load(path+'bwd_clas')
pred_bwd,lbl_bwd = learn_bwd.get_preds(ordered=True)
final_pred = (pred_fwd+pred_bwd)/2
accuracy(final_pred, lbl_fwd)

"""

"\nlearn = text_classifier_learner(data_clas, AWD_LSTM, pretrained=False)\nlearn.load(path+'fwd_clas')\npred_fwd,lbl_fwd = learn.get_preds(ordered=True)\naccuracy(pred_fwd, lbl_fwd)\n#\nlearn_bwd = text_classifier_learner(data_clas_bwd, AWD_LSTM, pretrained=False)\nlearn_bwd.load(path+'bwd_clas')\npred_bwd,lbl_bwd = learn_bwd.get_preds(ordered=True)\nfinal_pred = (pred_fwd+pred_bwd)/2\naccuracy(final_pred, lbl_fwd)\n\n"